<a href="https://colab.research.google.com/github/sensharma/adlproject/blob/main/adlproj_pytorch_base_cnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive/data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
apple_strawberry.jpg	dogs.png		   LibriSpeech
basket			FashionMNIST		   MNIST
cifar-10-batches-py	hymenoptera_data	   text_dataset_test
cifar-10-python.tar.gz	imagenet_class_index.json


In [ ]:
import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F

import os

In [ ]:
TORCH_DATAPATH = os.path.join(os.getcwd(), 'gdrive', 'MyDrive', 'data')
TORCH_MODELPATH = os.path.join(os.getcwd(), 'gdrive', 'MyDrive', 'colabdrive', 'adlproject', 'saved_models')

In [ ]:
train_dataset = datasets.MNIST(root=TORCH_DATAPATH,
                               train=True,
                               download=True,
                               transform=transforms.Compose([
                                                             transforms.ToTensor()
                                                             ])
                               )

test_dataset = datasets.MNIST(root=TORCH_DATAPATH,
                              train=False,
                              download=True,
                              transform=transforms.Compose([
                                                            transforms.ToTensor()
                                                            ])
                              )

fm_train_dataset = datasets.FashionMNIST(root=TORCH_DATAPATH,
                                         train=True,
                                         download=True,
                                         transform=transforms.Compose([
                                                                       transforms.ToTensor()
                                                                       ])
                                         )

fm_test_dataset = datasets.FashionMNIST(root=TORCH_DATAPATH,
                                        train=False,
                                        download=True,
                                        transform=transforms.Compose([
                                                                      transforms.ToTensor()
                                                                      ])
                                        )

mnist_train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=128,
    shuffle=True,
    )

mnist_test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=128,
    shuffle=True,
    )

f_mnist_train_loader = torch.utils.data.DataLoader(
    dataset=fm_train_dataset,
    batch_size=128,
    shuffle=True,
    )

f_mnist_test_loader = torch.utils.data.DataLoader(
    dataset=fm_test_dataset,
    batch_size=128,
    shuffle=True,
    )

In [ ]:
class Net(nn.Module):
    def __init__(self, h=28, w=28, n_channels=1):
        super(Net, self).__init__()
        self.h = h
        self.w = w
        self.n_channels = n_channels

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(16, 16), stride=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=(9, 9), stride=(1, 1)),
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(1600, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def x_compat(self, x):
        # print(f"x - type {type(x)}")
        if type(x) is not torch.Tensor:
            x_tensor = torch.from_numpy(x)
        else:
            x_tensor = x
        x_out = x_tensor.clone().reshape(x.shape[0],
                                         self.n_channels,
                                         self.h,
                                         self.w)
        return x_out.float()

    def forward(self, x):
        if x.ndim != 4:
            # print("in compat")
            x = self.x_compat(x)
        x = self.conv_layers(x)
        # print(f"shape: {x.shape}")
        x = x.view(-1, 1600)
        x = self.fc_layers(x)
        return x

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # loss = F.nll_loss(output, target)
        loss = F.nll_loss(output.log(), target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return loss

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output.log(), target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
    return correct

In [ ]:
num_epochs = 25
use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

best = 0
# train and save best model (based on validation accuracy)
for epoch in range(1, num_epochs + 1):
    loss = train(model, device, mnist_train_loader, optimizer, epoch)
    correct = test(model, device, mnist_test_loader)
    if correct > best:
        best = correct
        print(f'Saving for epoch: {epoch}')
        torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'loss': loss,
        }, f'{TORCH_MODELPATH}/mnist_checkpoint')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312877
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.867773
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.746521
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.371563
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.370412

Test set: Average loss: 0.0029, Accuracy: 8914/10000 (89%)

Saving for epoch: 1
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.432197
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.464964
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.421234
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.249867
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.263857

Test set: Average loss: 0.0017, Accuracy: 9306/10000 (93%)

Saving for epoch: 2
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.283971
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.331609
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.230216
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.269904
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.216681

Test set: Average loss: 0.0013, Accuracy: 9486/10000 (95%)

Saving for epoch: 3
Train Epoch: 4 [0/6000